<a href="https://colab.research.google.com/github/landcelita/LBMNNs/blob/master/LBMNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pygrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.6 MB 4.8 MB/s 
     |████████████████████████████████| 7.8 MB 80.7 MB/s 


In [2]:
import numpy as np
import torch
import torch.nn as nn
import os
import time
import datetime as dt
import pygrib
from datetime import timedelta
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torch.multiprocessing as multiprocessing

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
class InputLayer(nn.Module):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        
    def forward(self, u_vert, u_hori, rho):
        # use feq as input field
        # todo: make this layer be also learnable
        
        # feq(x,v) = C[v] * rho * (1 + 3 v.u + 4.5 (v.u)^2 - 1.5 u^2)
        feq = torch.empty((3, 3, self.height, self.width), dtype=torch.float32, device=device)
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        u_squared = u_vert ** 2 + u_hori ** 2
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + 3.0 * v_dot_u + 4.5 * v_dot_u * v_dot_u - 1.5 * u_squared)
                
        return feq

In [5]:
# test for InputLayer
# # Later, I'll check the immutability of the InputLayer
#  - it must put out the same tensor even after the weights are updated
def test_for_InputLayer():
    input_layer_u_vert = torch.tensor([[0.1]], dtype=torch.float32, device=device)
    input_layer_u_hori = torch.tensor([[-0.3]], dtype=torch.float32, device=device)
    input_layer_rho = torch.tensor([[0.8]], dtype=torch.float32, device=device)

    input_layer = InputLayer(1, 1).to(device)
    feq = input_layer(input_layer_u_vert, input_layer_u_hori, input_layer_rho)
    print(torch.sum(feq[2,:,0,0] - feq[0,:,0,0]) / torch.sum(feq[:,:,0,0])) # u_vert
    print(torch.sum(feq[:,2,0,0] - feq[:,0,0,0]) / torch.sum(feq[:,:,0,0])) # u_hori
    print(torch.sum(feq[:,:,0,0])) # rho

test_for_InputLayer()

tensor(0.1000, device='cuda:0')
tensor(-0.3000, device='cuda:0')
tensor(0.8000, device='cuda:0')


In [6]:
class StreamingLayer(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w0 = nn.Parameter(torch.zeros((in_height-2, in_width-2), dtype=torch.float32, device=device))
        self.w1 = nn.Parameter(torch.ones((in_height-2, in_width-2), dtype=torch.float32, device=device))
        
    def forward(self, f_prev):
        # f(x,v) = w0(x,v) + w1(x,v) * f_prev(x-v,v)
        
        f = torch.empty((3, 3, self.in_height-2, self.in_width-2), dtype=torch.float32, device=device)
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                f[dr+1,dc+1,:,:] = self.w0 + self.w1 * f_prev[dr+1,dc+1,1-dr:self.in_height-1-dr,1-dc:self.in_width-1-dc]
        
        return f
                

In [7]:
# test for StreamingLayer
def test_for_StreamingLayer():
    input_f_prev = torch.zeros((3, 3, 3, 3), dtype=torch.float64, device=device)
    input_f_prev[2,2,0,0] = 1.0
    input_f_prev[2,1,0,1] = 2.0
    input_f_prev[2,0,0,2] = 3.0
    input_f_prev[1,2,1,0] = 4.0
    input_f_prev[1,1,1,1] = 5.0
    input_f_prev[1,0,1,2] = 6.0
    input_f_prev[0,2,2,0] = 7.0
    input_f_prev[0,1,2,1] = 8.0
    input_f_prev[0,0,2,2] = 9.0
    print(input_f_prev)

    streaming_layer = StreamingLayer(3, 3).to(device)
    f_streamed = streaming_layer(input_f_prev)
    print(f_streamed)

test_for_StreamingLayer()

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 9.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 8., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [7., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 6.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 5., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [4., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 3.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 2., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], device='cuda:0', dtype=torch.float64)
tensor([[[[9.]],

         [[8.]],

         [[7.]]],


        [[[6.]],

         [[5.]],

         [[4.]]],


        [[[3.]],

         [[2.]],

         [[1.]]]], device='cuda:0', grad_fn=<CopySlices>)


In [8]:
class CollidingLayer(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w1 = nn.Parameter(torch.full((in_height, in_width), fill_value=3.0, dtype=torch.float32, device=device), requires_grad=True)
        self.w2 = nn.Parameter(torch.full((in_height, in_width), fill_value=0.0, dtype=torch.float32, device=device), requires_grad=True)
        self.w3 = nn.Parameter(torch.full((in_height, in_width), fill_value=4.5, dtype=torch.float32, device=device), requires_grad=True)
        self.w4 = nn.Parameter(torch.full((in_height, in_width), fill_value=-1.5, dtype=torch.float32, device=device), requires_grad=True)
        
    def forward(self, f_prev):
        # feq(x,v) = C(v) * rho(x) * (1 + w1(x,v) v.u(x) + w2(x,v) vXu(x) + w3(x,v) (v.u(x))^2 + w4(x,v) u(x)^2)
        # f = (f_prev + feq) / 2
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        
        rho = torch.sum(f_prev, dim=(0,1))
        u_vert = (f_prev[2,0,:,:] + f_prev[2,1,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[0,1,:,:] - f_prev[0,2,:,:]) / rho
        u_hori = (f_prev[0,2,:,:] + f_prev[1,2,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[1,0,:,:] - f_prev[2,0,:,:]) / rho
        u_squared = u_vert ** 2 + u_hori ** 2
        feq = torch.empty((3, 3, self.in_height, self.in_width), dtype=torch.float32, device=device)
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                vXu = dr * u_hori - dc * u_vert
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + (self.w1 + self.w3 * v_dot_u) * v_dot_u + self.w2 * vXu + self.w4 * u_squared)
        
        return (f_prev + feq) / 2.0

In [9]:
# test for CollidingLayer
def test_for_CollidingLayer():
    input_f = torch.tensor([[[[9., 9.]],[[8., 8.]],[[7., 7.]]],[[[6., 6.]],[[5., 5.]],[[4., 4.]]],[[[3., 3.]],[[2., 2.]],[[1., 1.]]]], dtype=torch.float32, device=device)
    colliding_layer = CollidingLayer(1, 2)
    collided_field = colliding_layer(input_f)

    print(torch.sum(collided_field[:,:,0,0])) # rho 45.0
    print((torch.sum(collided_field[2,:,0,0]) - torch.sum(collided_field[0,:,0,0])) / torch.sum(collided_field[:,:,0,0])) # u_vert
    print((1.+2.+3.-9.-8.-7.)/45.)
    print((torch.sum(collided_field[:,2,0,0]) - torch.sum(collided_field[:,0,0,0])) / torch.sum(collided_field[:,:,0,0])) # u_hori
    print((1.+4.+7.-3.-6.-9.)/45.)

test_for_CollidingLayer()

tensor(45., device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.4000, device='cuda:0', grad_fn=<DivBackward0>)
-0.4
tensor(-0.1333, device='cuda:0', grad_fn=<DivBackward0>)
-0.13333333333333333


In [10]:
class OutputLayer(nn.Module):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        
    def forward(self, f):
        rho = torch.sum(f, dim=(0,1))
        u_vert = (f[2,0,:,:] + f[2,1,:,:] + f[2,2,:,:] - f[0,0,:,:] - f[0,1,:,:] - f[0,2,:,:]) / rho
        u_hori = (f[0,2,:,:] + f[1,2,:,:] + f[2,2,:,:] - f[0,0,:,:] - f[1,0,:,:] - f[2,0,:,:]) / rho
        return u_vert, u_hori, rho

In [11]:
class LBM(nn.Module):
    margin = 4
    
    def __init__(self, height, width):
        super(LBM, self).__init__()
        self.input_layer = InputLayer(height, width)
        self.streaming1_layer = StreamingLayer(height, width)
        self.colliding1_layer = CollidingLayer(height-2, width-2)
        self.streaming2_layer = StreamingLayer(height-2, width-2)
        self.colliding2_layer = CollidingLayer(height-4, width-4)
        self.streaming3_layer = StreamingLayer(height-4, width-4)
        self.colliding3_layer = CollidingLayer(height-6, width-6)
        self.streaming4_layer = StreamingLayer(height-6, width-6)
        self.output_layer = OutputLayer(height-8, width-8)
        
    def forward(self, u_vert, u_hori, rho):
        f = self.input_layer(u_vert, u_hori, rho)
        f = self.streaming1_layer(f)
        f = self.colliding1_layer(f)
        f = self.streaming2_layer(f)
        f = self.colliding2_layer(f)
        f = self.streaming3_layer(f)
        f = self.colliding3_layer(f)
        f = self.streaming4_layer(f)
        u_vert_out, u_hori_out, rho_out = self.output_layer(f)
        return u_vert_out, u_hori_out, rho_out

In [26]:
def loss_func(u_vert_pred, u_hori_pred, rho_pred, u_vert_target, u_hori_target, rho_target):
    # MSE
    return torch.sum(torch.square(u_vert_pred - u_vert_target) + torch.square(u_hori_pred - u_hori_target)) * 10000. +\
        torch.sum(torch.square(rho_pred - rho_target)) * 200.

In [13]:
class WeatherDatasetLazy(torch.utils.data.Dataset):
    def __init__(self, src_dir, datetimes, transform, target_transform, margin):
        self.src_dir = src_dir
        self.input_paths = []
        self.target_paths = []
        self.margin = margin # this value is the number of streaming layers that reduce the border cells.
        # eg. ...                            xxx
        #     ...  --(1 Streaming Layer)-->  x.x
        #     ...                            xxx
        for datetime in datetimes:
            self.input_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
            datetime += timedelta(hours=3)
            self.target_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.input_paths)
    
    def __getitem__(self, idx):
        input_grib = pygrib.open(self.input_paths[idx])
        rho = torch.from_numpy(np.array(input_grib.select()[0].values, dtype=np.float32)).to(device)
        u_hori = torch.from_numpy(np.array(input_grib.select()[1].values, dtype=np.float32)).to(device)
        u_vert = torch.from_numpy(np.array(input_grib.select()[2].values, dtype=np.float32)).to(device)
        target_grib = pygrib.open(self.target_paths[idx])
        rho_target = torch.from_numpy(np.array(target_grib.select()[0].values, dtype=np.float32)).to(device)
        u_hori_target = torch.from_numpy(np.array(target_grib.select()[1].values, dtype=np.float32)).to(device)
        u_vert_target = torch.from_numpy(np.array(target_grib.select()[2].values, dtype=np.float32)).to(device)
        u_vert, u_hori, rho = self.transform(u_vert, u_hori, rho)
        u_vert_target, u_hori_target, rho_target = self.target_transform(u_vert_target, u_hori_target, rho_target, self.margin)
        return {
            "u_vert": u_vert,
            "u_hori": u_hori,
            "rho": rho,
            "u_vert_target": u_vert_target,
            "u_hori_target": u_hori_target,
            "rho_target": rho_target
        }

In [14]:
class WeatherDataset(torch.utils.data.Dataset):
    def __init__(self, src_dir, datetimes, transform, target_transform, margin):
        self.src_dir = src_dir
        self.input_paths = []
        self.target_paths = []
        self.margin = margin # this value is the number of streaming layers that reduce the border cells.
        # eg. ...                            xxx
        #     ...  --(1 Streaming Layer)-->  x.x
        #     ...                            xxx
        for datetime in datetimes:
            self.input_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
            datetime += timedelta(hours=3)
            self.target_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
        self.rhos = []
        self.u_horis = []
        self.u_verts = []
        self.rho_targets = []
        self.u_hori_targets = []
        self.u_vert_targets = []
        self.transform = transform
        self.target_transform = target_transform
        now = time.time()
        for idx in range(len(self.input_paths)):
            if (idx+1) % 100 == 0:
                print(f'{idx+1} samples collected, {time.time() - now:.2f} s')
                now = time.time()
            input_grib = pygrib.open(self.input_paths[idx])
            rho = torch.from_numpy(np.array(input_grib.select()[0].values, dtype=np.float32)).to(device)
            u_hori = torch.from_numpy(np.array(input_grib.select()[1].values, dtype=np.float32)).to(device)
            u_vert = torch.from_numpy(np.array(input_grib.select()[2].values, dtype=np.float32)).to(device)
            target_grib = pygrib.open(self.target_paths[idx])
            rho_target = torch.from_numpy(np.array(target_grib.select()[0].values, dtype=np.float32)).to(device)
            u_hori_target = torch.from_numpy(np.array(target_grib.select()[1].values, dtype=np.float32)).to(device)
            u_vert_target = torch.from_numpy(np.array(target_grib.select()[2].values, dtype=np.float32)).to(device)
            self.rhos.append(rho)
            self.u_horis.append(u_hori)
            self.u_verts.append(u_vert)
            self.rho_targets.append(rho_target)
            self.u_hori_targets.append(u_hori_target)
            self.u_vert_targets.append(u_vert_target)
    
    def __len__(self):
        return len(self.input_paths)
    
    def __getitem__(self, idx):
        u_vert, u_hori, rho = self.transform(self.u_verts[idx], self.u_horis[idx], self.rhos[idx])
        u_vert_target, u_hori_target, rho_target = self.target_transform(self.u_vert_targets[idx], self.u_hori_targets[idx], self.rho_targets[idx], self.margin)
        return {
            "u_vert": u_vert,
            "u_hori": u_hori,
            "rho": rho,
            "u_vert_target": u_vert_target,
            "u_hori_target": u_hori_target,
            "rho_target": rho_target
        }

In [24]:
def transform(u_vert, u_hori, rho):
    # 下向き正
    return u_vert * -0.01, u_hori * 0.01, rho * 0.0001

def target_transform(u_vert, u_hori, rho, margin):
    if margin == 0:
        return u_vert * -0.01, u_hori * 0.01, rho * 0.0001
    else:
        return u_vert[margin:-margin, margin:-margin] * -0.01, u_hori[margin:-margin, margin:-margin] * 0.01, rho[margin:-margin, margin:-margin] * 0.0001

In [17]:
# test for WeatherDataset
def test_for_weatherdataset():
    datetimeset = [dt.datetime(2011, 7, 1), dt.datetime(2011, 7, 2)]
    grib = pygrib.open('/content/drive/MyDrive/lab_data/data/2011070103.grib2')
    u_vert = np.array(grib.select()[2].values, dtype=np.float32)
    src_dir = '/content/drive/MyDrive/lab_data/data/'
    weather_dataset = WeatherDataset(src_dir, datetimeset, transform, target_transform, 1)
    u_vert_direct = torch.from_numpy(u_vert).to(device)
    u_vert_target = weather_dataset.__getitem__(0)['u_vert_target']
    print(weather_dataset.__len__()) # 2
    print(torch.equal(u_vert_target, u_vert_direct[1:-1, 1:-1] * -0.01)) # True
    print(u_vert_direct.size())

test_for_weatherdataset()

2
True
torch.Size([505, 481])


In [18]:
def make_dataset_and_dataloader():
    print('Creating a dataset and dataloader...')
    data_dir = '/content/drive/MyDrive/lab_data/data'
    datetimeset = [dt.datetime(2011+i, 7, 1, 0) + timedelta(days=j) for i in range(10) for j in range(92)]
    dataset = WeatherDataset(data_dir, datetimeset, transform, target_transform, LBM.margin)
    batch_size = 1
    train_split = 0.8
    shuffle_dataset = True # if false, this model will learn only by the 'past' data, and forcast the 'future' data.
    random_seed = 42
    
    dataset_size =  len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(train_split * dataset_size))
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, valid_indices = indices[:split], indices[split:]
    
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(valid_indices)
    
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=0)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=0)

    return dataset, train_loader, valid_loader

dataset, train_loader, valid_loader = make_dataset_and_dataloader()

Creating a dataset and dataloader...
100 samples collected, 64.33 s
200 samples collected, 65.66 s
300 samples collected, 64.08 s
400 samples collected, 66.79 s
500 samples collected, 64.46 s
600 samples collected, 64.07 s
700 samples collected, 63.72 s
800 samples collected, 65.35 s
900 samples collected, 65.31 s


In [27]:
# todo: enable the model to receive batch data
def main(dataset, train_loader, valid_loader):
    print('Creating the model...')
    lbm = LBM(505, 481).to(device) # got by the prev cell
    
    num_epochs = 200
    total_time = 0.0
    
    optimizer = torch.optim.Adam(lbm.parameters())
    
    print("optimizer = " + str(optimizer))
    print("max_epochs = %3d " % num_epochs)
    
    print('Training the model...')
    lbm.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_started_time = time.time()
        for (batch_idx, batch) in enumerate(train_loader):
            optimizer.zero_grad()
            u_vert_out, u_hori_out, rho_out = lbm(batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze())
            loss_val = loss_func(u_vert_out, u_hori_out, rho_out, batch['u_vert_target'].squeeze(), batch['u_hori_target'].squeeze(), batch['rho_target'].squeeze())
            epoch_loss += loss_val.item()
            loss_val.backward()
            # if batch_idx % 100 == 0:
            #     print(f'batch: {batch_idx}')
            optimizer.step()
        
        print("epoch = %4d  loss = %.4e  epoch time = %0.2fs" % (epoch, epoch_loss, time.time() - epoch_started_time))
    print('Done')
    
    print('Computing the model accuracy')
    lbm.eval()
        
main(dataset, train_loader, valid_loader)

Creating the model...
optimizer = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
max_epochs = 200 
Training the model...
epoch =    0  loss = 1.1076e+09  epoch time = 27.09s
epoch =    1  loss = 1.0717e+09  epoch time = 26.71s
epoch =    2  loss = 1.0596e+09  epoch time = 26.71s
epoch =    3  loss = 1.0519e+09  epoch time = 26.69s
epoch =    4  loss = 1.0465e+09  epoch time = 26.76s
epoch =    5  loss = 1.0413e+09  epoch time = 26.70s
epoch =    6  loss = 1.0377e+09  epoch time = 27.14s
epoch =    7  loss = 1.0333e+09  epoch time = 27.37s
epoch =    8  loss = 1.0297e+09  epoch time = 27.11s
epoch =    9  loss = 1.0253e+09  epoch time = 27.00s
epoch =   10  loss = 1.0229e+09  epoch time = 26.80s
epoch =   11  loss = 1.0196e+09  epoch time = 26.78s
epoch =   12  loss = 1.0162e+09  epoch time = 26.74s
epoch =   13  l